# Enhancing RAG with Knowledge Graphs

### Query Process Workflow with Llama-Index and Nebula Graph:

  - **Build Knowledge Graph for the given Context**
  - **Get Key Entities/Relationships related to Query**
  - **Get SubGraphs**
  - **Generate answer based on SubGraphs**

In [52]:
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-azure-openai
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-azure-openai
%pip install ipython-ngql nebula3-python

In [53]:
#!pip uninstall llama_index==0.8.9
!pip install pydantic==1.10.13
!pip install llama-index

In [54]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [55]:
import os
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  

In [56]:

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# For Azure OpenAI
api_version = "2023-07-01-preview"
api_key = "e------------------------------------"
azure_endpoint = "https://--------------------.openai.azure.com/"


lc_llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="gpt-35-turbo",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [57]:
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ['GRAPHD_HOST'] = "127.0.0.1"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally

space_name = "rag_workshop"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [58]:
%load_ext ngql
connection_string = f"--address {os.environ['GRAPHD_HOST']} --port 9669 --user root --password {os.environ['NEBULA_PASSWORD']}"
%ngql {connection_string}

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql
Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,rag_workshop


In [59]:
%ngql CREATE SPACE IF NOT EXISTS rag_workshop(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [60]:
%%ngql
USE rag_workshop;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [61]:
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [62]:
%ngql SHOW HOSTS

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Host,Port,Status,Leader count,Leader distribution,Partition distribution,Version
0,storaged0,9779,ONLINE,0,No valid partition,No valid partition,3.6.0
1,storaged1,9779,ONLINE,0,No valid partition,No valid partition,3.6.0
2,storaged2,9779,ONLINE,1,rag_workshop:1,rag_workshop:1,3.6.0


In [63]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore
from llama_index.core import Settings
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex,  VectorStoreIndex
from llama_index.core import ServiceContext, set_global_service_context


Settings.llm = lc_llm 
Settings.embed_model = embedding_llm
Settings.chunk_size = 512

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [64]:
from llama_index.core import download_loader

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Tesla Cybertruck"], auto_suggest=False
)

In [65]:
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings

In [66]:
kg_index.storage_context.persist(persist_dir='./storage_graph')

In [67]:
storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
service_context = ServiceContext.from_defaults(
    llm=lc_llm,
    embed_model=embedding_llm,
)

set_global_service_context(service_context)

kg_index = load_index_from_storage(
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

INFO:llama_index.core.indices.loading:Loading all indices.


/tmp/ipykernel_59476/3725880670.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [68]:
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

nl2kg_query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=lc_llm,
)

In [69]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

In [70]:
from IPython.display import Markdown

response_graph_rag = kg_index_query_engine.query("What is Cybertruck")

print("\n")
display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Extracted keywords: ['Cybertruck']
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 72e2755e-92c4-444f-9fdc-0ca9512c05ba: === Powertrain ===
Tesla stated that they use a platform approach to the powe...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 3e4146a4-cfbf-4502-b18b-318f6cb118c7: == Production and availability ==

Cybertruck production began in Gigafactory...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: e7e4cd1d-3ad1-4c84-adae-c6a0aed42030: === 2019 concept ===

The concept Cybertruck was unveiled in Los Angeles in N...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Cybertruck{name: Cyber

<b>Cybertruck is a vehicle developed by Tesla that uses a platform approach to powertrain components, including a single permanent magnet motor rotor/stator design, a single induction motor rotor/stator design, a single motor inverter design, and a single gear set design. It comes in three configurations: tri-motor AWD, dual-motor AWD, and single-motor RWD, with power ranging from 845 hp (630 kW) on the tri-motor down to 315 hp (235 kW) in the single-motor version. The windows use borosilicate glass, and the vehicle has raised safety concerns due to its angular design and stiff stainless-steel exterior. Cybertruck production began in Gigafactory Texas in Austin, Texas, with pre-production models in July 2023, and serial production had begun by November 2023. As of December 2023, Tesla confirmed that the Cybertruck will be available exclusively in the United States, Canada, and Mexico, with no plans for release in other global markets, including Europe and Australia.</b>

In [71]:
response_graph_rag = kg_index_query_engine.query("Who is Elon Musk?")

print("\n")
display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Extracted keywords: ['Musk', 'Elon Musk', 'Elon']
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 5a6f2b44-3f19-4f28-b54b-c6e637f10f98: == Design ==


=== Inspiration and styling ===

According to Musk, the design...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: c4520e68-7cfb-44a2-87d8-f18af5fbcb8b: === Reservations ===

Beginning in November 2019, Tesla accepted Cybertruck r...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Musk{name: Musk} -[relationship:{relationship: Claimed}]-> Windows were damaged because of sledgehammer demonstration{name: Windows were damaged because of sledgehammer demonstration} <-[relationship:{rel

<b>Elon Musk is mentioned multiple times in the provided context information as the CEO of Tesla and the person involved in the design process of the Cybertruck. He was inspired by Blade Runner and the Lotus Esprit driven by James Bond in The Spy Who Loved Me, which doubled as a submarine, for the design of the Cybertruck. Musk's son's quote "Why doesn't the future look like the future?" was used as an inspiration for the design as well. Musk wanted the Cybertruck to have all the utility of a pick-up truck but drive like a sports car. He also claimed that final specifications and pricing for the Cybertruck would be materially different from those unveiled on the concept vehicle in 2019.</b>

In [72]:
response_graph_rag = kg_index_query_engine.query("How powerful is the Cybertruck?")
print("\n")
display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Extracted keywords: ['Cybertruck', 'powerful']
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 72e2755e-92c4-444f-9fdc-0ca9512c05ba: === Powertrain ===
Tesla stated that they use a platform approach to the powe...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 3e4146a4-cfbf-4502-b18b-318f6cb118c7: == Production and availability ==

Cybertruck production began in Gigafactory...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: e7e4cd1d-3ad1-4c84-adae-c6a0aed42030: === 2019 concept ===

The concept Cybertruck was unveiled in Los Angeles in N...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Cybertruck

<b>The Cybertruck comes in three configurations: tri-motor AWD, dual-motor AWD, and single-motor RWD. The tri-motor AWD version offers a maximum combined output of 845 hp (630 kW) in Beast Mode, split as 276 hp (206 kW) for the front motor and 284 hp (212 kW) for each rear motor. The dual-motor AWD version has a total power output of 600 hp (450 kW), with an induction motor on the front axle with a maximum output of 303 hp (226 kW) and a permanent magnet motor on the rear axle with a maximum output of 297 hp (221 kW). The single-motor RWD version has a power output of 315 hp (235 kW).</b>

In [73]:
#re=nl2kg_query_engine.query("SHOW HOSTS")

In [74]:
#response = nl2kg_query_engine.query(
    #"Tell me about Peter Quill?",
#)
#display(Markdown(f"<b>{response}</b>"))